# Conferindo os dados

In [0]:
display(dbutils.fs.ls("mnt/dados/inbound"))

In [0]:
path = "dbfs:/mnt/dados/inbound/dados_brutos_imoveis.json"

In [0]:
dados = spark.read.json(path)

In [0]:
display(dados)

# Transformando os dados para a camada bronze

## Removendo as colunas requisitadas

In [0]:
dados = dados.drop("imagens", "usuario")

In [0]:
dados.show()

## Criando uma coluna de identificaçáo

In [0]:
from pyspark.sql.functions import col

In [0]:
df_bronze = dados.withColumn("id", col("anuncio.id"))
display(df_bronze)

## Salvando na camada bronze

In [0]:
path_save = "dbfs:/mnt/dados/bronze/dataset_imoveis"
df_bronze.write.format("delta").mode("overwrite").save(path_save)

# Transformando os dados para a camada Silver

In [0]:
path_bronze = "dbfs:/mnt/dados/bronze/dataset_imoveis/"

In [0]:
df = spark.read.format("delta").load(path_bronze)
df.show()

## Transformando cada campo para uma coluna

In [0]:
display(df.select("anuncio.*"))

In [0]:
display(
    df.select("anuncio.*", "anuncio.endereco.*")
)

In [0]:
dados_detalhados = df.select("anuncio.*", "anuncio.endereco.*")

In [0]:
dados_detalhados.printSchema()

## Removendo colunas requisitadas

In [0]:
dados_detalhados = dados_detalhados.drop("caracteristicas", "endereco")
display(dados_detalhados)

In [0]:
path_save = "dbfs:/mnt/dados/silver/dataset_imoveis"
dados_detalhados.write.format("delta").mode("overwrite").save(path_save)